In [1]:
import numpy as np
import pandas as pd
m2 = pd.read_csv('merged_data.csv', dtype={"CLNT_ID":np.str, "SESS_ID":np.str},thousands=",")
#CLNT 그대로
CUSM2=pd.read_csv('data_regression.csv',dtype={'CLNT_ID':str})
CUSM2=CUSM2.loc[:,'CLNT_ID']
m2=pd.concat([CUSM2, m2], axis=1)

In [2]:
from sklearn.preprocessing import MinMaxScaler,RobustScaler,MaxAbsScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [5]:
F_678 = m2.loc[(m2['GENDER_F']==1)]
F_678 = F_678.loc[(F_678['CLNT_AGE']==60) | (F_678['CLNT_AGE']==70) | (F_678['CLNT_AGE']==80)]  
F_678['CLNT_AGE'].value_counts()

60    3784
70     541
80      23
Name: CLNT_AGE, dtype: int64

In [6]:
F678ID=F_678['CLNT_ID']
del F_678['CLNT_ID']
F_678.head()

,HITS_SEQ,PD_BUY_AM,PD_BUY_CT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,CLNT_AGE,GENDER_F,GENDER_M,weekend,DVC_desktop,...,CLAC0_0,CLAC0_1,CLAC0_2,CLAC0_3,CLAC0_4,CLAC0_5,CLAC0_6,CLAC0_7,CLAC0_8,CLAC0_9
107,85.000000,72000.000000,1.0,94.000000,1507.000000,60,1,0,2,0.0,...,0,0,0,2.0,0.0,0,0,0.0,0.0,0.0
221,45.500000,21250.000000,1.0,77.500000,911.500000,60,1,0,0,0.0,...,0,0,0,0.0,0.0,0,0,0.0,2.0,0.0
307,125.000000,37450.000000,1.0,124.000000,1118.000000,60,1,0,0,0.0,...,0,0,0,0.0,0.0,0,0,0.0,2.0,0.0
425,56.000000,14400.000000,1.0,49.000000,1169.000000,60,1,0,2,0.0,...,0,0,0,0.0,0.0,0,0,0.0,0.0,2.0
555,47.333333,123666.666667,1.0,66.666667,1547.666667,60,1,0,0,0.0,...,0,0,0,0.0,2.0,0,0,0.0,1.0,0.0


In [7]:
#대대분류 삭제
F_678_drop = F_678.drop(F_678.loc[:, F_678.columns.str.startswith('CLAC0')], axis = 1)
F_678_drop.columns
#mms = MinMaxScaler()
mms = MaxAbsScaler()
#mms = RobustScaler()
mms.fit(F_678_drop) #대대분류 뺀 column들만 갖고
F_678_transformed = mms.transform(F_678_drop)

In [8]:
k=3
km = KMeans(n_clusters=k, algorithm='auto')
km.fit(F_678_transformed)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [9]:
F_678_transformed = pd.DataFrame(F_678_transformed)
F_678_transformed.columns = m2.columns[1:35]
F_678_transformed.head()

,HITS_SEQ,PD_BUY_AM,PD_BUY_CT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,CLNT_AGE,GENDER_F,GENDER_M,weekend,DVC_desktop,...,ZON_Jeollabuk-do,ZON_Jeollanam-do,ZON_Seoul,ZON_Ulsan,month_4,month_5,month_6,month_7,month_8,month_9
0,0.178947,0.026087,0.017857,0.188377,0.092109,0.75,1.0,0.0,0.003578,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00813
1,0.095789,0.007699,0.017857,0.155311,0.055712,0.75,1.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00813
2,0.263158,0.013569,0.017857,0.248497,0.068333,0.75,1.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.006135,0.000000,0.0,0.00000
3,0.117895,0.005217,0.017857,0.098196,0.071450,0.75,1.0,0.0,0.003578,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00813
4,0.099649,0.044807,0.017857,0.133601,0.094595,0.75,1.0,0.0,0.000000,0.0,...,0.0,0.0,0.016304,0.0,0.0,0.0,0.003067,0.003077,0.0,0.00000


In [10]:
F_678_transformed['predict'] = km.predict(F_678_transformed)
F_678_transformed['predict'].value_counts()

0    2804
1    1272
2     272
Name: predict, dtype: int64

In [11]:
F_678_transformed.index = F_678.index
F_678_transformed['CLNT_ID']=m2['CLNT_ID']
F_678_transformed.head()

,HITS_SEQ,PD_BUY_AM,PD_BUY_CT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,CLNT_AGE,GENDER_F,GENDER_M,weekend,DVC_desktop,...,ZON_Seoul,ZON_Ulsan,month_4,month_5,month_6,month_7,month_8,month_9,predict,CLNT_ID
107,0.178947,0.026087,0.017857,0.188377,0.092109,0.75,1.0,0.0,0.003578,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00813,1,0000886
221,0.095789,0.007699,0.017857,0.155311,0.055712,0.75,1.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00813,0,0001778
307,0.263158,0.013569,0.017857,0.248497,0.068333,0.75,1.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.006135,0.000000,0.0,0.00000,1,0002559
425,0.117895,0.005217,0.017857,0.098196,0.071450,0.75,1.0,0.0,0.003578,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00813,0,0003373
555,0.099649,0.044807,0.017857,0.133601,0.094595,0.75,1.0,0.0,0.000000,0.0,...,0.016304,0.0,0.0,0.0,0.003067,0.003077,0.0,0.00000,0,0004604


In [ ]:
def scorefun(k, x, data, transformed_data, category) :

    #k : clustering 개수
    #new_data : new data
    #data : F_20 처럼 m2에서 나이와 성별만 지정한 데이터. 대대분류 포함됨
    #transformed_data : F_20_transformed 처럼 성별과 나이대를 지정하고, predict column이 추가된 transforemd data
    #category : score를 원하는 원하는 대대분류
    
    category_rate = np.empty([k,10])
    for kk in range(k) :
       # sum_of_sum = data[['CLAC0_0', 'CLAC0_1', 'CLAC0_2', 'CLAC0_3', 'CLAC0_4', 'CLAC0_5','CLAC0_6', 'CLAC0_7', 'CLAC0_8', 'CLAC0_9']].sum().sum()
        idx=np.where(transformed_data['predict']==kk)

        clustering = data.loc[data.index[idx]] 
        sum_of_sum = clustering[['CLAC0_0', 'CLAC0_1', 'CLAC0_2', 'CLAC0_3', 'CLAC0_4', 'CLAC0_5','CLAC0_6', 'CLAC0_7', 'CLAC0_8', 'CLAC0_9']].sum().sum()
        category_rate[kk] = clustering[['CLAC0_0', 'CLAC0_1', 'CLAC0_2', 'CLAC0_3', 'CLAC0_4', 'CLAC0_5','CLAC0_6', 'CLAC0_7', 'CLAC0_8', 'CLAC0_9']].sum()/sum_of_sum

    
    distance_from_centers = [0] * k
    xdata=transformed_data.loc[x, :"month_9"]
    for i in range(k) :
        distance_from_centers[i] = np.sqrt(sum((km.cluster_centers_[i] - xdata)**2))
        
    result = [0] * k
    for i in range(k) :
        result[i] = (category_rate[i][category]/distance_from_centers[i])
    
    return sum(result)*100

In [ ]:
F_678_transformed.index=F_678_transformed['CLNT_ID']
F_678.index=F_678_transformed['CLNT_ID']

In [ ]:
# 두 인덱스가 같아야 함!
F_678_transformed.index, F_678.index

In [ ]:
F_678['furniture_SCORE'] = F_678_transformed['CLNT_ID'].apply(lambda x: scorefun(3, x, F_678, F_678_transformed,0))
F_678['travel_SCORE'] = F_678_transformed['CLNT_ID'].apply(lambda x: scorefun(3, x, F_678, F_678_transformed,5))
F_678['electronic_SCORE'] = F_678_transformed['CLNT_ID'].apply(lambda x: scorefun(3, x, F_678, F_678_transformed,1))
F_678['pet_SCORE'] = F_678_transformed['CLNT_ID'].apply(lambda x: scorefun(3, x, F_678, F_678_transformed,2))
F_678['sports_SCORE'] = F_678_transformed['CLNT_ID'].apply(lambda x: scorefun(3, x, F_678, F_678_transformed,3))
F_678['food_SCORE'] = F_678_transformed['CLNT_ID'].apply(lambda x: scorefun(3, x, F_678, F_678_transformed,4))
F_678['travel_SCORE'] = F_678_transformed['CLNT_ID'].apply(lambda x: scorefun(3, x, F_678, F_678_transformed,5))
F_678['baby_SCORE'] = F_678_transformed['CLNT_ID'].apply(lambda x: scorefun(3, x, F_678, F_678_transformed,6))
F_678['kitchen_SCORE'] = F_678_transformed['CLNT_ID'].apply(lambda x: scorefun(3, x, F_678, F_678_transformed,7))
F_678['fashion_SCORE'] = F_678_transformed['CLNT_ID'].apply(lambda x: scorefun(3, x, F_678, F_678_transformed,8))
F_678['beauty_SCORE'] = F_678_transformed['CLNT_ID'].apply(lambda x: scorefun(3, x, F_678, F_678_transformed,9))

In [ ]:
F_678.head()

In [ ]:
#Save
F_678[['furniture_SCORE', 'electronic_SCORE', 'pet_SCORE', 'sports_SCORE',
       'food_SCORE', 'travel_SCORE', 'baby_SCORE', 'kitchen_SCORE',
       'fashion_SCORE', 'beauty_SCORE']].to_csv('F_678.csv',index=False)

In [3]:
#Load
F_678_Score=pd.read_csv('F_678.csv',dtype={'CUSID':str})
F_678_Score.head()

,CUSID,furniture_SCORE,electronic_SCORE,pet_SCORE,sports_SCORE,food_SCORE,travel_SCORE,baby_SCORE,kitchen_SCORE,fashion_SCORE,beauty_SCORE
0,107,115.817373,109.770399,117.197957,150.751696,133.789086,-3.324774,135.265252,133.309637,175.338739,146.951006
1,221,178.384131,170.249938,181.553154,229.683760,207.423725,51.969504,205.937221,205.106503,263.823178,227.796945
2,307,124.894384,118.187853,126.169424,163.318592,143.691227,-17.342850,146.590429,143.990599,190.624243,157.746750
3,425,226.745479,216.882633,231.178765,291.056336,264.086780,87.572558,260.911060,260.688452,333.010682,289.984376
4,555,194.755539,186.025120,198.336406,250.529154,226.545250,62.874386,224.614053,223.949808,287.375299,248.771965


In [12]:
F_678_transformed.index=F_678_Score.index
F_678_Score=pd.concat([F_678_transformed['CLNT_ID'], F_678_Score], axis=1)

In [13]:
#Ratio
m2['PD_BUY_CT2']=m2['CLAC0_0']+m2['CLAC0_1']+m2['CLAC0_2']+m2['CLAC0_3']+m2['CLAC0_4']+m2['CLAC0_4']+m2['CLAC0_5']+m2['CLAC0_6']+m2['CLAC0_7']+m2['CLAC0_8']+m2['CLAC0_9']
m2['C0_Ratio']=m2['CLAC0_0']/m2['PD_BUY_CT2']
m2['C1_Ratio']=m2['CLAC0_1']/m2['PD_BUY_CT2']
m2['C2_Ratio']=m2['CLAC0_2']/m2['PD_BUY_CT2']
m2['C3_Ratio']=m2['CLAC0_3']/m2['PD_BUY_CT2']
m2['C4_Ratio']=m2['CLAC0_4']/m2['PD_BUY_CT2']
m2['C5_Ratio']=m2['CLAC0_5']/m2['PD_BUY_CT2']
m2['C6_Ratio']=m2['CLAC0_6']/m2['PD_BUY_CT2']
m2['C7_Ratio']=m2['CLAC0_7']/m2['PD_BUY_CT2']
m2['C8_Ratio']=m2['CLAC0_8']/m2['PD_BUY_CT2']
m2['C9_Ratio']=m2['CLAC0_9']/m2['PD_BUY_CT2']

In [14]:
m3=pd.merge(m2,F_678_Score,on=['CLNT_ID'],how='inner')
m3.columns.tolist()

['CLNT_ID',
 'HITS_SEQ',
 'PD_BUY_AM',
 'PD_BUY_CT',
 'TOT_PAG_VIEW_CT',
 'TOT_SESS_HR_V',
 'CLNT_AGE',
 'GENDER_F',
 'GENDER_M',
 'weekend',
 'DVC_desktop',
 'DVC_mobile',
 'DVC_tablet',
 'ZON_Busan',
 'ZON_Chungcheongbuk-do',
 'ZON_Chungcheongnam-do',
 'ZON_Daegu',
 'ZON_Daejeon',
 'ZON_Gangwon-do',
 'ZON_Gwangju',
 'ZON_Gyeonggi-do',
 'ZON_Gyeongsangbuk-do',
 'ZON_Gyeongsangnam-do',
 'ZON_Incheon',
 'ZON_Jeju-do',
 'ZON_Jeollabuk-do',
 'ZON_Jeollanam-do',
 'ZON_Seoul',
 'ZON_Ulsan',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'CLAC0_0',
 'CLAC0_1',
 'CLAC0_2',
 'CLAC0_3',
 'CLAC0_4',
 'CLAC0_5',
 'CLAC0_6',
 'CLAC0_7',
 'CLAC0_8',
 'CLAC0_9',
 'PD_BUY_CT2',
 'C0_Ratio',
 'C1_Ratio',
 'C2_Ratio',
 'C3_Ratio',
 'C4_Ratio',
 'C5_Ratio',
 'C6_Ratio',
 'C7_Ratio',
 'C8_Ratio',
 'C9_Ratio',
 'CUSID',
 'furniture_SCORE',
 'electronic_SCORE',
 'pet_SCORE',
 'sports_SCORE',
 'food_SCORE',
 'travel_SCORE',
 'baby_SCORE',
 'kitchen_SCORE',
 'fashion_SCORE',
 'beaut

In [15]:
m4=m3[[
 'CLNT_ID',
 'PD_BUY_CT2',
 'CLAC0_0',
 'CLAC0_1',
 'CLAC0_2',
 'CLAC0_3',
 'CLAC0_4',
 'CLAC0_5',
 'CLAC0_6',
 'CLAC0_7',
 'CLAC0_8',
 'CLAC0_9',
 'C0_Ratio',
 'C1_Ratio',
 'C2_Ratio',
 'C3_Ratio',
 'C4_Ratio',
 'C5_Ratio',
 'C6_Ratio',
 'C7_Ratio',
 'C8_Ratio',
 'C9_Ratio',
 'furniture_SCORE',
 'travel_SCORE',
 'electronic_SCORE',
 'pet_SCORE',
 'sports_SCORE',
 'food_SCORE',
 'baby_SCORE',
 'kitchen_SCORE',
 'fashion_SCORE',
 'beauty_SCORE']]

In [ ]:
import tqdm
m4['furniture_new']=arange(0,len(m4))
for i in tqdm.tqdm(arange(len(m4))) :
    if m4['C0_Ratio'][i] == 0 :
        m4['furniture_new'][i]=m4['furniture_SCORE'][i]
    else : m4['furniture_new'][i]=m4['furniture_SCORE'][i]*(1+m4['C0_Ratio'][i])

In [ ]:
m4[['furniture_new','CLAC0_0']].sort_values(['furniture_new'],ascending=False)

In [ ]:
m4['travel_new']=arange(0,len(m4))
for i in arange(len(m4)) :
    if m4['C1_Ratio'][i] == 0 :
        m4['travel_new'][i]=m4['travel_SCORE'][i]
    else : m4['travel_new'][i]=m4['travel_SCORE'][i]*(1+m4['C1_Ratio'][i])

In [ ]:
m4[['travel_new','CLAC0_1']].sort_values(['travel_new'],ascending=False)

In [ ]:
m4['electronic_new']=arange(0,len(m4))
for i in arange(len(m4)) :
    if m4['C2_Ratio'][i] == 0 :
        m4['electronic_new'][i]=m4['electronic_SCORE'][i]
    else : m4['electronic_new'][i]=m4['electronic_SCORE'][i]*(1+m4['C2_Ratio'][i])

In [ ]:
m4[['electronic_new','CLAC0_2']].sort_values(['electronic_new'],ascending=False)

In [ ]:
m4['pet_new']=arange(0,len(m4))
for i in arange(len(m4)) :
    if m4['C3_Ratio'][i] == 0 :
        m4['pet_new'][i]=m4['pet_SCORE'][i]
    else : m4['pet_new'][i]=m4['pet_SCORE'][i]*(1+m4['C3_Ratio'][i])

In [ ]:
m4[['pet_new','CLAC0_3']].sort_values(['pet_new'],ascending=False)

In [ ]:
m4['sports_new']=arange(0,len(m4))
for i in arange(len(m4)) :
    if m4['C4_Ratio'][i] == 0 :
        m4['sports_new'][i]=m4['sports_SCORE'][i]
    else : m4['sports_new'][i]=m4['sports_SCORE'][i]*(1+m4['C4_Ratio'][i])

In [ ]:
m4[['sports_new','CLAC0_4']].sort_values(['sports_new'],ascending=False)

In [ ]:
m4['food_new']=arange(0,len(m4))
for i in arange(len(m4)) :
    if m4['C5_Ratio'][i] == 0 :
        m4['food_new'][i]=m4['food_SCORE'][i]
    else : m4['food_new'][i]=m4['food_SCORE'][i]*(1+m4['C5_Ratio'][i])

In [ ]:
m4[['food_new','CLAC0_5']].sort_values(['food_new'],ascending=False)

In [ ]:
m4['baby_new']=arange(0,len(m4))
for i in arange(len(m4)) :
    if m4['C6_Ratio'][i] == 0 :
        m4['baby_new'][i]=m4['baby_SCORE'][i]
    else : m4['baby_new'][i]=m4['baby_SCORE'][i]*(1+m4['C6_Ratio'][i])

In [ ]:
m4[['baby_new','CLAC0_6']].sort_values(['baby_new'],ascending=False)

In [ ]:
m4['kitchen_new']=arange(0,len(m4))
for i in arange(len(m4)) :
    if m4['C7_Ratio'][i] == 0 :
        m4['kitchen_new'][i]=m4['kitchen_SCORE'][i]
    else : m4['kitchen_new'][i]=m4['kitchen_SCORE'][i]*(1+m4['C7_Ratio'][i])

In [ ]:
m4[['kitchen_new','CLAC0_7']].sort_values(['kitchen_new'],ascending=False)

In [ ]:
m4['fashion_new']=arange(0,len(m4))
for i in arange(len(m4)) :
    if m4['C8_Ratio'][i] == 0 :
        m4['fashion_new'][i]=m4['fashion_SCORE'][i]
    else : m4['fashion_new'][i]=m4['fashion_SCORE'][i]*(1+m4['C8_Ratio'][i])

In [ ]:
m4[['fashion_new','CLAC0_8']].sort_values(['fashion_new'],ascending=False)

In [ ]:
m4['beauty_new']=arange(0,len(m4))
for i in arange(len(m4)) :
    if m4['C9_Ratio'][i] == 0 :
        m4['beauty_new'][i]=m4['beauty_SCORE'][i]
    else : m4['beauty_new'][i]=m4['beauty_SCORE'][i]*(1+m4['C9_Ratio'][i])

In [ ]:
m4[['beauty_new','CLAC0_9']].sort_values(['beauty_new'],ascending=False)

In [ ]:
F_678_Score=m4[['furniture_new','travel_new','electronic_new','pet_new','sports_new','food_new','baby_new','kitchen_new','fashion_new','beauty_new']]
F_678_Score.mean().values

In [ ]:
labels=np.array(labels)
stats=F_678_Score.loc[20,['furniture_new','travel_new','electronic_new','pet_new','sports_new','food_new','baby_new','kitchen_new','fashion_new','beauty_new']].values

In [ ]:
mean_stats=F_678_Score.mean()[0:].values
mean_stats=np.concatenate((mean_stats,[mean_stats[0]]))

In [ ]:
angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)
# close the plot
stats=np.concatenate((stats,[stats[0]]))
angles=np.concatenate((angles,[angles[0]]))

In [ ]:
stats2=F_678_Score.loc[70,['furniture_new','travel_new','electronic_new','pet_new','sports_new','food_new','baby_new','kitchen_new','fashion_new','beauty_new']].values

stats2=np.concatenate((stats2,[stats2[0]]))

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111, polar=True)
ax.plot(angles, stats, 'o-', linewidth=2)
ax.fill(angles, stats, alpha=0.25)
ax.set_thetagrids(angles * 180/np.pi, labels)
#ax.set_title([F_678_Score.loc[2,"Name"]])
ax.plot(angles, mean_stats, 'o-', linewidth=2)
ax.fill(angles, mean_stats,"b",alpha=0.1)
#ax.plot(angles, median_stats, 'o-', linewidth=2)
#ax.fill(angles, median_stats,alpha=0.1)

ax.grid(True)

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111, polar=True)
ax.plot(angles, stats2, 'o-', linewidth=2)
ax.fill(angles, stats2, alpha=0.25)
ax.set_thetagrids(angles * 180/np.pi, labels)
#ax.set_title([F_678_Score.loc[2,"Name"]])
ax.plot(angles, mean_stats, 'o-', linewidth=2)
ax.fill(angles, mean_stats,"b",alpha=0.1)
#ax.plot(angles, median_stats, 'o-', linewidth=2)
#ax.fill(angles, median_stats,alpha=0.1)

ax.grid(True)